In [1]:
library(haven)
library(dplyr)
library(lavaan)
library(semTools)

# Load the Phase 2 dataset with ordered items
dat <- readRDS("data/processed/analysis_phase1_ordered.rds")

# Convert group vars from haven_labelled -> factor (using value labels)
dat <- dat %>%
  mutate(
    gender           = haven::as_factor(gender, levels = "labels"),
    academic_year    = haven::as_factor(academic_year, levels = "labels"),
    dept_group       = haven::as_factor(dept_group, levels = "labels"),
    living_situation = haven::as_factor(living_situation, levels = "labels")
  )

# Remove rows with missing levels per grouping when running each MI
drop_na_group <- function(df, g) dplyr::filter(df, !is.na(.data[[g]]))

# DASS CFA representation (3-factor), used for MI
model_dass_cfa <- '
Dep =~ dQ3D + dQ5D + dQ10D + dQ13D + dQ16D + dQ17D + dQ21D
Anx =~ dQ2A + dQ4A + dQ7A + dQ9A + dQ15A + dQ19A + dQ20A
Str =~ dQ1S + dQ6S + dQ8S + dQ11S + dQ12S + dQ14S + dQ18S
'

# DASS ordered indicators
dass_items <- c("dQ1S","dQ2A","dQ3D","dQ4A","dQ5D","dQ6S","dQ7A","dQ8S","dQ9A",
                "dQ10D","dQ11S","dQ12S","dQ13D","dQ14S","dQ15A","dQ16D","dQ17D",
                "dQ18S","dQ19A","dQ20A","dQ21D")

# BRS one-factor model (scored orientation)
model_brs <- 'BRS =~ rQ1 + rQ3 + rQ5 + rQ2_r + rQ4_r + rQ6_r'
brs_items <- c("rQ1","rQ3","rQ5","rQ2_r","rQ4_r","rQ6_r")

# Common options for ordinal MI
mi_opts <- list(
  parameterization = "theta",
  ID.fac = "std.lv",
  ID.cat = "Wu.Estabrook.2016",
  estimator = "WLSMV",
  return.fit = TRUE
)



Attaching package: ‘dplyr’

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union

This is lavaan 0.6-19
lavaan is FREE software! Please report any bugs.
 
###############################################################################
This is semTools 0.5-7
All users of R (or SEM) are invited to submit functions or ideas for functions.
###############################################################################


In [4]:
run_mi <- function(model, data, ordered_items, group_var, label_prefix){
  # ensure no missing group labels
  df <- dplyr::filter(data, !is.na(.data[[group_var]]))

  # Common settings for all fits
  common <- list(
    parameterization = "theta",
    ID.fac = "std.lv",
    ID.cat = "Wu.Estabrook.2016",
    estimator = "WLSMV",
    return.fit = TRUE
  )

  # CONFIGURAL
  cfg <- semTools::measEq.syntax(
    configural.model = model,
    data     = df,
    ordered  = ordered_items,
    group    = group_var,
    group.equal = c(),
    parameterization = common$parameterization,
    ID.fac   = common$ID.fac,
    ID.cat   = common$ID.cat,
    estimator= common$estimator,
    return.fit = common$return.fit
  )

  # THRESHOLDS
  thr <- semTools::measEq.syntax(
    configural.model = model,
    data     = df,
    ordered  = ordered_items,
    group    = group_var,
    group.equal = c("thresholds"),
    parameterization = common$parameterization,
    ID.fac   = common$ID.fac,
    ID.cat   = common$ID.cat,
    estimator= common$estimator,
    return.fit = common$return.fit
  )

  # THRESHOLDS + LOADINGS (ordinal "metric")
  met <- semTools::measEq.syntax(
    configural.model = model,
    data     = df,
    ordered  = ordered_items,
    group    = group_var,
    group.equal = c("thresholds","loadings"),
    parameterization = common$parameterization,
    ID.fac   = common$ID.fac,
    ID.cat   = common$ID.cat,
    estimator= common$estimator,
    return.fit = common$return.fit
  )

  comp <- semTools::compareFit(cfg, thr, met)

  dir.create("outputs/MI", showWarnings = FALSE, recursive = TRUE)
  sink(file.path("outputs/MI", paste0(label_prefix, "_compare.txt")))
  print(comp)
  cat("\n\n-- Configural fit --\n"); print(fitMeasures(cfg, c("cfi","tli","rmsea","srmr","chisq","df")))
  cat("\n-- Thresholds fit --\n"); print(fitMeasures(thr, c("cfi","tli","rmsea","srmr","chisq","df")))
  cat("\n-- Thresholds+Loadings fit --\n"); print(fitMeasures(met, c("cfi","tli","rmsea","srmr","chisq","df")))
  sink()

  list(cfg = cfg, thr = thr, met = met, comp = comp)
}


In [5]:
mi_dass_gender  <- run_mi(model_dass_cfa, dat, dass_items, "gender",           "DASS_gender")
mi_dass_year    <- run_mi(model_dass_cfa, dat, dass_items, "academic_year",    "DASS_academic_year")
mi_dass_dept    <- run_mi(model_dass_cfa, dat, dass_items, "dept_group",       "DASS_dept_group")
mi_dass_livesit <- run_mi(model_dass_cfa, dat, dass_items, "living_situation", "DASS_living_situation")


Warning messages:
1: lavaan->lav_object_post_check():  
   covariance matrix of latent variables is not positive definite in group 2; use lavInspect(fit, 
   "cov.lv") to investigate. 
2: lavaan->lav_object_post_check():  
   covariance matrix of latent variables is not positive definite in group 2; use lavInspect(fit, 
   "cov.lv") to investigate. 
3: lavaan->lav_object_post_check():  
   covariance matrix of latent variables is not positive definite in group 2; use lavInspect(fit, 
   "cov.lv") to investigate. 
4: lavaan->lav_object_post_check():  
   covariance matrix of latent variables is not positive definite in group 3; use lavInspect(fit, 
   "cov.lv") to investigate. 
Warning messages:
1: lavaan->lav_model_vcov():  
   The variance-covariance matrix of the estimated parameters (vcov) does not appear to be positive 
   definite! The smallest eigenvalue (= -4.095564e-17) is smaller than zero. This may be a symptom 
   that the model is not identified. 
2: lavaan->lav_object_post

In [6]:
mi_brs_gender  <- run_mi(model_brs, dat, brs_items, "gender",           "BRS_gender")
mi_brs_year    <- run_mi(model_brs, dat, brs_items, "academic_year",    "BRS_academic_year")
mi_brs_dept    <- run_mi(model_brs, dat, brs_items, "dept_group",       "BRS_dept_group")
mi_brs_livesit <- run_mi(model_brs, dat, brs_items, "living_situation", "BRS_living_situation")


In [8]:
delta_report <- function(comp_obj){
  # Pull fit measures from compareFit
  tryCatch({
    as.data.frame(comp_obj)
  }, error = function(e) {
    capture.output(print(comp_obj), file = tempfile()) # fallback
    NULL
  })
}

rep_gender_dass   <- delta_report(mi_dass_gender$comp)
rep_year_dass     <- delta_report(mi_dass_year$comp)
rep_dept_dass     <- delta_report(mi_dass_dept$comp)
rep_livesit_dass  <- delta_report(mi_dass_livesit$comp)

rep_gender_brs    <- delta_report(mi_brs_gender$comp)
rep_year_brs      <- delta_report(mi_brs_year$comp)
rep_dept_brs      <- delta_report(mi_brs_dept$comp)
rep_livesit_brs   <- delta_report(mi_brs_livesit$comp)

dir.create("outputs/MI/tables", showWarnings = FALSE, recursive = TRUE)
if(!is.null(rep_gender_dass)) write.csv(rep_gender_dass,  "outputs/MI/tables/compare_DASS_gender.csv", row.names = FALSE)
if(!is.null(rep_year_dass))   write.csv(rep_year_dass,    "outputs/MI/tables/compare_DASS_academic_year.csv", row.names = FALSE)
if(!is.null(rep_dept_dass))   write.csv(rep_dept_dass,    "outputs/MI/tables/compare_DASS_dept_group.csv", row.names = FALSE)
if(!is.null(rep_livesit_dass))write.csv(rep_livesit_dass, "outputs/MI/tables/compare_DASS_living_situation.csv", row.names = FALSE)

if(!is.null(rep_gender_brs))  write.csv(rep_gender_brs,   "outputs/MI/tables/compare_BRS_gender.csv", row.names = FALSE)
if(!is.null(rep_year_brs))    write.csv(rep_year_brs,     "outputs/MI/tables/compare_BRS_academic_year.csv", row.names = FALSE)
if(!is.null(rep_dept_brs))    write.csv(rep_dept_brs,     "outputs/MI/tables/compare_BRS_dept_group.csv", row.names = FALSE)
if(!is.null(rep_livesit_brs)) write.csv(rep_livesit_brs,  "outputs/MI/tables/compare_BRS_living_situation.csv", row.names = FALSE)


In [17]:
library(ggplot2); library(dplyr); library(tidyr)

extract_fit <- function(fits_list, label){
  tibble::tibble(
    Level = c("Configural","Thresholds","Thresh+Load"),
    cfi   = c(fitMeasures(fits_list$cfg, "cfi"),
              fitMeasures(fits_list$thr, "cfi"),
              fitMeasures(fits_list$met, "cfi")),
    tli   = c(fitMeasures(fits_list$cfg, "tli"),
              fitMeasures(fits_list$thr, "tli"),
              fitMeasures(fits_list$met, "tli")),
    rmsea = c(fitMeasures(fits_list$cfg, "rmsea"),
              fitMeasures(fits_list$thr, "rmsea"),
              fitMeasures(fits_list$met, "rmsea")),
    srmr  = c(fitMeasures(fits_list$cfg, "srmr"),
              fitMeasures(fits_list$thr, "srmr"),
              fitMeasures(fits_list$met, "srmr")),
    Grouping = label
  )
}

# Example: DASS across gender/year/dept/living
fit_gender <- extract_fit(mi_dass_gender,  "Gender")
fit_year   <- extract_fit(mi_dass_year,    "Academic year")
fit_dept   <- extract_fit(mi_dass_dept,    "Dept group")
fit_live   <- extract_fit(mi_dass_livesit, "Living situation")

fits_all <- bind_rows(fit_gender, fit_year, fit_dept, fit_live) |>
  pivot_longer(cols = c(cfi,tli,rmsea,srmr), names_to = "Index", values_to = "Value")

p_fit <- ggplot(fits_all, aes(Level, Value, fill = Index)) +
  geom_col(position = position_dodge(width = 0.7), width = 0.6) +
  facet_wrap(~ Grouping, ncol = 2) +
  geom_hline(yintercept = 0.95, linetype = 2, color = "gray50", data = subset(fits_all, Index %in% c("cfi","tli"))) +
  geom_hline(yintercept = 0.06, linetype = 2, color = "gray50", data = subset(fits_all, Index == "rmsea")) +
  geom_hline(yintercept = 0.08, linetype = 2, color = "gray70", data = subset(fits_all, Index == "srmr")) +
  theme_minimal(base_size = 11) +
  labs(title = "Ordinal MI fit across levels", x = "", y = "Value")
ggsave("outputs/MI/plot_MI_fit_levels_DASS.png", p_fit, width = 10, height = 7, dpi = 200,bg="white")


Warning messages:
1: `geom_hline()`: Ignoring `data` because `yintercept` was provided. 
2: `geom_hline()`: Ignoring `data` because `yintercept` was provided. 
3: `geom_hline()`: Ignoring `data` because `yintercept` was provided. 


In [18]:
delta_fits <- function(fits_list, label){
  get <- function(x, m) fitMeasures(x, m)
  data.frame(
    Grouping = label,
    dCFI_thr  = get(fits_list$thr, "cfi")  - get(fits_list$cfg, "cfi"),
    dCFI_met  = get(fits_list$met, "cfi")  - get(fits_list$thr, "cfi"),
    dRMSEA_thr= get(fits_list$thr, "rmsea")- get(fits_list$cfg, "rmsea"),
    dRMSEA_met= get(fits_list$met, "rmsea")- get(fits_list$thr, "rmsea")
  )
}

deltas <- bind_rows(
  delta_fits(mi_dass_gender,  "Gender"),
  delta_fits(mi_dass_year,    "Academic year"),
  delta_fits(mi_dass_dept,    "Dept group"),
  delta_fits(mi_dass_livesit, "Living situation")
)

readr::write_csv(deltas, "outputs/MI/MI_deltas_DASS.csv")

d_long <- deltas |>
  pivot_longer(-Grouping, names_to = "Change", values_to = "Delta")

p_delta <- ggplot(d_long, aes(Change, Delta, fill = Grouping)) +
  geom_col(position = position_dodge(width = 0.7), width = 0.6) +
  geom_hline(yintercept = -0.01, linetype = 2, color = "gray50") +
  geom_hline(yintercept =  0.01, linetype = 2, color = "gray50") +
  theme_minimal(base_size = 11) +
  labs(title = "ΔCFI and ΔRMSEA across MI levels (DASS)", x = "", y = "Delta")
ggsave("outputs/MI/plot_MI_deltas_DASS.png", p_delta, width = 9, height = 5, dpi = 200 ,bg="white")


In [19]:
std_by_group <- function(fit, group_var){
  # Extract standardized solution with group column
  ss <- standardizedSolution(fit)  # includes 'group' index
  # Map group indices to labels
  grp_levels <- lavInspect(fit, "group.label")
  ss$Group <- factor(ss$group, labels = grp_levels)
  subset(ss, op == "=~")[, c("Group","lhs","rhs","est.std")]
}

# Example for DASS across gender: use the thresholds+loadings fit
ss_gender <- std_by_group(mi_dass_gender$met, "gender")
load_mat <- ss_gender |>
  rename(Factor = lhs, Item = rhs, Loading = est.std)

# Heatmap of item loadings by group and factor
p_load_heat <- ggplot(load_mat, aes(Item, Group, fill = Loading)) +
  geom_tile() +
  facet_wrap(~ Factor, ncol = 1) +
  scale_fill_gradient2(low = "#ef4444", mid = "white", high = "#22c55e", midpoint = 0.5) +
  theme_minimal(base_size = 10) +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5)) +
  labs(title = "Standardized loadings by group (DASS; thresholds+loadings)", x = "Item", y = "")
ggsave("outputs/MI/plot_loadings_bygroup_DASS_gender.png", p_load_heat, width = 12, height = 7, dpi = 200,bg="white")


In [20]:
thresh_by_group <- function(fit){
  pe <- parameterEstimates(fit, standardized = FALSE)
  grp_levels <- lavInspect(fit, "group.label")
  pe$Group <- factor(pe$group, labels = grp_levels)
  # Threshold rows are of the form "Item|t1", op="|"
  subset(pe, op == "|")[, c("Group","lhs","rhs","est")]
}

tb_gender <- thresh_by_group(mi_dass_gender$met)

# Compute per-item SD across groups as a quick instability index
thr_var <- tb_gender |>
  group_by(lhs, rhs) |>
  summarize(sd_est = sd(est, na.rm = TRUE), .groups = "drop")

p_thr <- ggplot(thr_var, aes(x = interaction(lhs, rhs), y = sd_est)) +
  geom_col(fill = "#6366f1") +
  theme_minimal(base_size = 10) +
  theme(axis.text.x = element_text(angle = 90, vjust = 0.5)) +
  labs(title = "Threshold variability across groups (DASS; thresholds+loadings)", x = "Item|threshold", y = "SD across groups")
ggsave("outputs/MI/plot_threshold_variability_DASS_gender.png", p_thr, width = 12, height = 5, dpi = 200,bg="white")


In [21]:
# Example: fit bars for BRS across all groupings
fit_gender_brs <- extract_fit(mi_brs_gender,  "Gender")
fit_year_brs   <- extract_fit(mi_brs_year,    "Academic year")
fit_dept_brs   <- extract_fit(mi_brs_dept,    "Dept group")
fit_live_brs   <- extract_fit(mi_brs_livesit, "Living situation")

fits_brs <- bind_rows(fit_gender_brs, fit_year_brs, fit_dept_brs, fit_live_brs) |>
  pivot_longer(c(cfi,tli,rmsea,srmr), names_to = "Index", values_to = "Value")

p_fit_brs <- ggplot(fits_brs, aes(Level, Value, fill = Index)) +
  geom_col(position = position_dodge(width = 0.7), width = 0.6) +
  facet_wrap(~ Grouping, ncol = 2) +
  theme_minimal(base_size = 11) +
  labs(title = "Ordinal MI fit across levels (BRS)", x = "", y = "Value")
ggsave("outputs/MI/plot_MI_fit_levels_BRS.png", p_fit_brs, width = 10, height = 7, dpi = 200,bg="white")
